In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-nature-conservancy-fisheries-monitoring/train.zip
/kaggle/input/the-nature-conservancy-fisheries-monitoring/test_stg2.7z
/kaggle/input/the-nature-conservancy-fisheries-monitoring/test_stg1.zip
/kaggle/input/the-nature-conservancy-fisheries-monitoring/sample_submission_stg2.csv.zip
/kaggle/input/the-nature-conservancy-fisheries-monitoring/sample_submission_stg1.csv.zip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import os
import cv2
import csv
import shutil
from glob import glob
from PIL import Image
from IPython.display import FileLink

In [3]:
print(os.listdir("../input/the-nature-conservancy-fisheries-monitoring/"))

['train.zip', 'test_stg2.7z', 'test_stg1.zip', 'sample_submission_stg2.csv.zip', 'sample_submission_stg1.csv.zip']


In [4]:
!unzip ../input/the-nature-conservancy-fisheries-monitoring/train.zip
!unzip ../input/the-nature-conservancy-fisheries-monitoring/sample_submission_stg1.csv
!unzip ../input/the-nature-conservancy-fisheries-monitoring/sample_submission_stg2.csv

Archive:  ../input/the-nature-conservancy-fisheries-monitoring/train.zip
   creating: train/
  inflating: train/.DS_Store         
   creating: __MACOSX/
   creating: __MACOSX/train/
  inflating: __MACOSX/train/._.DS_Store  
   creating: train/ALB/
  inflating: train/ALB/img_00003.jpg  
  inflating: train/ALB/img_00010.jpg  
  inflating: train/ALB/img_00012.jpg  
  inflating: train/ALB/img_00015.jpg  
  inflating: train/ALB/img_00019.jpg  
  inflating: train/ALB/img_00020.jpg  
  inflating: train/ALB/img_00029.jpg  
  inflating: train/ALB/img_00032.jpg  
  inflating: train/ALB/img_00037.jpg  
  inflating: train/ALB/img_00038.jpg  
  inflating: train/ALB/img_00039.jpg  
  inflating: train/ALB/img_00041.jpg  
  inflating: train/ALB/img_00043.jpg  
  inflating: train/ALB/img_00045.jpg  
  inflating: train/ALB/img_00055.jpg  
  inflating: train/ALB/img_00057.jpg  
  inflating: train/ALB/img_00074.jpg  
  inflating: train/ALB/img_00085.jpg  
  inflating: train/ALB/img_00090.jpg  
  inflatin

In [5]:
os.mkdir('valid/')

In [6]:
os.mkdir('valid/ALB/')
os.mkdir('valid/BET/')
os.mkdir('valid/DOL/')
os.mkdir('valid/LAG/')
os.mkdir('valid/NoF/')
os.mkdir('valid/OTHER/')
os.mkdir('valid/SHARK/')
os.mkdir('valid/YFT/')

In [7]:
ALB_valid = glob('train/ALB/*.jpg')
shuf = np.random.permutation(ALB_valid)

for i in range(int(len(ALB_valid) / 10)): shutil.move(shuf[i], 'valid/ALB/')

In [8]:
BET_valid = glob('train/BET/*.jpg')
shuf = np.random.permutation(BET_valid)

for i in range(int(len(BET_valid) / 10)): shutil.move(shuf[i], 'valid/BET/')

In [9]:
DOL_valid = glob('train/DOL/*.jpg')
shuf = np.random.permutation(DOL_valid)

for i in range(int(len(DOL_valid) / 10)): shutil.move(shuf[i], 'valid/DOL/')

In [10]:
LAG_valid = glob('train/LAG/*.jpg')
shuf = np.random.permutation(LAG_valid)

for i in range(int(len(LAG_valid) / 10)): shutil.move(shuf[i], 'valid/LAG/')

In [11]:
NoF_valid = glob('train/NoF/*.jpg')
shuf = np.random.permutation(NoF_valid)

for i in range(int(len(NoF_valid) / 10)): shutil.move(shuf[i], 'valid/NoF/')

In [12]:
OTHER_valid = glob('train/OTHER/*.jpg')
shuf = np.random.permutation(OTHER_valid)

for i in range(int(len(OTHER_valid) / 10)): shutil.move(shuf[i], 'valid/OTHER/')

In [13]:
SHARK_valid = glob('train/SHARK/*.jpg')
shuf = np.random.permutation(SHARK_valid)

for i in range(int(len(SHARK_valid) / 10)): shutil.move(shuf[i], 'valid/SHARK/')

In [14]:
YFT_valid = glob('train/YFT/*.jpg')
shuf = np.random.permutation(YFT_valid)

for i in range(int(len(YFT_valid) / 10)): shutil.move(shuf[i], 'valid/YFT/')

In [15]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D, Conv2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras.models import model_from_json

In [16]:
batch_size = 32

In [17]:
datagen = image.ImageDataGenerator()
trn_batches = datagen.flow_from_directory('train/', target_size = (224, 224),
            class_mode = 'categorical', shuffle = True, batch_size = batch_size)

val_batches = datagen.flow_from_directory('valid/', target_size = (224, 224),
            class_mode = 'categorical', shuffle = True, batch_size = batch_size)

Found 3404 images belonging to 8 classes.
Found 373 images belonging to 8 classes.
